### ESMA 4016
### Regresion Logistica con las libreria H20  y scikit-learn varias clases
#### Edgar Acuna

In [17]:
import h2o
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
from h2o.estimators.glm   import H2OGeneralizedLinearEstimator

h2o.init()
h2o.no_progress()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,11 hours 21 mins
H2O cluster timezone:,America/La_Paz
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.6
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_edgar2017_7mwin8
H2O cluster total nodes:,1
H2O cluster free memory:,1.581 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


### I- Aplicando Regresion Logistica a  Vehicle  usando h20

In [18]:
vehicle = h2o.import_file("https://academic.uprm.edu/eacuna/vehicle.csv")
predictors=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18']
vehicle['class']=vehicle['class'].asfactor()
response_col="class"
# Create test/train split
#train, test = vehicle.split_frame(ratios=[0.75], seed=1)
glm_model = H2OGeneralizedLinearEstimator(family= "multinomial", lambda_ = 0)
glm_model.train(predictors, response_col, training_frame= vehicle)
glm_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1524120852419_159


ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.11191357107
RMSE: 0.33453485778
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-04-19 14:15:52,0.000 sec,0,1172.2577785,1.3856475
,2018-04-19 14:15:52,0.018 sec,1,491.0765863,0.5804688
,2018-04-19 14:15:52,0.031 sec,2,352.4440830,0.4166006
,2018-04-19 14:15:52,0.045 sec,3,309.2958840,0.3655980
,2018-04-19 14:15:52,0.059 sec,4,295.6671101,0.3494883
---,---,---,---,---,---
,2018-04-19 14:15:52,0.572 sec,46,283.8269327,0.3354928
,2018-04-19 14:15:52,0.584 sec,47,283.8233220,0.3354886
,2018-04-19 14:15:52,0.596 sec,48,283.8201451,0.3354848
,2018-04-19 14:15:52,0.608 sec,49,283.8172428,0.3354814



See the whole table with table.as_data_frame()


In [19]:
#Hallando la tasa de prediccion
y_pred=glm_model.predict(vehicle)
print (y_pred['predict']==vehicle['class']).sum()/float(len(vehicle))

0.83451536643


### II Regresion Logistica para vehicle usando scikit learn

In [20]:
df1=pd.read_csv("http://academic.uprm.edu/eacuna/vehicle.csv")
#Convirtiendo en matriz la tabla de predictoras y la columna de clases
y=df1['class']
X=df1.iloc[:,0:18]
#Haciendo la regresion logistica y calculando su precision
model = LogisticRegression()
model = model.fit(X, y)
model.score(X,y)
predictions = model.predict(X)
print(classification_report(y, predictions))

             precision    recall  f1-score   support

          1       0.93      0.97      0.95       218
          2       0.70      0.61      0.65       212
          3       0.67      0.71      0.69       217
          4       0.96      0.97      0.97       199

avg / total       0.81      0.82      0.81       846



### III. Regresion Logistica para  Landsat usando scikit-learn

In [21]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]
X=data.iloc[:,0:36]
#Haciendo la regresion logistica y calculando su precision
model = LogisticRegression()
model = model.fit(X, y)
model.score(X,y)
predictions = model.predict(X)
print(classification_report(y, predictions))

             precision    recall  f1-score   support

          1       0.96      0.99      0.97      1072
          2       0.88      0.96      0.92       479
          3       0.79      0.97      0.87       961
          4       0.64      0.13      0.21       415
          5       0.90      0.63      0.74       470
          6       0.77      0.90      0.83      1038

avg / total       0.83      0.84      0.82      4435



### Regresion Logistica para Landsat usando h20

In [27]:
landsat = h2o.import_file("https://academic.uprm.edu/eacuna/landsat.txt")
predictors=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35',"C36"]
landsat['C37']=landsat['C37'].asfactor()
response_col="C37"


In [28]:
glm_model = H2OGeneralizedLinearEstimator(family= "multinomial", lambda_ = 0)
glm_model.train(predictors, response_col, training_frame= landsat)
glm_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1524120852419_161


ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.0944183823545
RMSE: 0.307275743193
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-04-19 14:23:04,0.000 sec,0,7603.4530902,1.7144201
,2018-04-19 14:23:04,0.061 sec,1,3305.5076038,0.7453230
,2018-04-19 14:23:04,0.131 sec,2,1903.4490374,0.4291881
,2018-04-19 14:23:04,0.189 sec,3,1570.8020589,0.3541831
,2018-04-19 14:23:04,0.241 sec,4,1439.6670264,0.3246149
---,---,---,---,---,---
,2018-04-19 14:23:05,1.408 sec,27,1354.4232603,0.3053942
,2018-04-19 14:23:05,1.459 sec,28,1354.4200992,0.3053935
,2018-04-19 14:23:05,1.508 sec,29,1354.4176999,0.3053929
,2018-04-19 14:23:05,1.558 sec,30,1354.4160414,0.3053926



See the whole table with table.as_data_frame()


In [29]:
#Hallando la tasa de prediccion
y_pred=glm_model.predict(landsat)
print (y_pred['predict']==landsat['C37']).sum()/float(len(landsat))

0.880721533258
